<a href="https://colab.research.google.com/github/akimotolab/Policy_Optimization_Tutorial/blob/main/3_actor_critic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 準備

まず，必要なパッケージのインストールとインポート，および仮想displayを設定します．

In [1]:
# 必要なパッケージのインストール
!apt update
!pip install swig
!apt install xvfb
!pip install pyvirtualdisplay
!pip install gymnasium[box2d]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,242 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:12 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy/main Sources [2,244 kB]
Get:13 http://archive.ubuntu.com/ubuntu ja

In [2]:
from pyvirtualdisplay import Display
import torch

# 仮想ディスプレイの設定
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

In [3]:
import random
import numpy as np
from scipy.special import softmax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import gymnasium as gym
from IPython import display

続いて，第１回の資料で定義した`rollout`などの基本的な関数をここでも定義しておきます．

In [4]:
def rollout(envname, policy=None, render=False, seed=None):
    if render:
        env = gym.make(envname, render_mode="rgb_array")
    else:
        env = gym.make(envname)
    history = []
    img = []

    # 乱数の設定
    if seed is not None:
        random.seed(int(seed))
    envseed = random.randint(0, 1000)
    actseed = random.randint(0, 1000)
    observation, info = env.reset(seed=envseed)
    env.action_space.seed(actseed)

    # 可視化用の設定
    if render:
        d = Display()
        d.start()
        img.append(env.render())

    # メインループ（環境とのインタラクション）
    terminated = False
    truncated = False
    while not (terminated or truncated):

        # 行動を選択
        if policy is None:
            action = env.action_space.sample()
        else:
            action = policy(observation)

        # 行動を実行
        next_observation, reward, terminated, truncated, info = env.step(action)
        history.append([observation, action, next_observation, reward, terminated, truncated, info])
        observation = next_observation
        if render:
            display.clear_output(wait=True)
            img.append(env.render())
    env.close()
    return history, img


def visualize(img):
    dpi = 72
    interval = 50
    plt.figure(figsize=(img[0].shape[1]/dpi, img[0].shape[0]/dpi), dpi=dpi)
    patch = plt.imshow(img[0])
    plt.axis=('off')
    animate = lambda i: patch.set_data(img[i])
    ani = animation.FuncAnimation(plt.gcf(), animate, frames=len(img), interval=interval)
    display.display(display.HTML(ani.to_jshtml()))
    plt.close()


def cumulative_reward(history):
    return sum(hist[3] for hist in history)

# Actor-Critic法による方策最適化（online & on-policy）

今回は「Actor-Critic法」を見ていきます．
第２回は，価値関数をモンテカルロ近似する方策勾配法であるREINFORCEアルゴリズムを紹介しました．
今回紹介するActor-Critic法も方策勾配法の一種ですが，価値関数の推定にTD誤差（temporal-difference誤差）を活用する点が大きく異なります．

もう一点，前回扱ったREINFORCEとの違いがあります．REINFORCEでは，エピソード毎に方策勾配を計算し，方策や状態価値を更新していました．今回扱うActor-Critic法もそのように扱うことができますが，ここではこれに加えて，各エピソードの中の各ステップ（状態遷移）毎に方策を更新していく方向を見ていきます．
このようなアプローチ（エピソード内に方策を学習していくアプローチ）をオンラインアプローチと言います．

加えて，Actor-Critic法は，REINFORCE（モンテカルロ法）では必ずしも適切ではない非エピソディックタスク（連続タスク）に対しても適用可能です．
これについても見ていきます．

## 状態価値の再帰表現

まず，状態価値のおさらいです．
状態$s$の価値を，「$s_0 = s$からインタラクションを始めて，方策$\pi$に従って行動選択した際に得られる割引累積報酬の期待値$\mathrm{E}[G_0 \mid s_0 = s]$」と定義します．
これを$V^{\pi}(s)$と書きます．
定義からわかるように，状態価値は方策$\pi$に依存しています．
割引累積報酬が
$$
G_{t} = r_{t+1} + \gamma G_{t+1}
$$
という再帰的な関係式を満たすことを考えると，状態価値は
$$
V^{\pi}(s) = \mathrm{E}[r_{t+1} + \gamma V^{\pi}(s_{t+1}) \mid s_t = s]
$$
という関係式を満たすことがわかります．

## TD誤差を用いた価値関数の推定

方策勾配を計算するには価値を推定することが必要になります．
REINFORCEアルゴリズムでは，価値をモンテカルロ近似していました．
すなわち，1エピソード分，現在の方策を用いて環境とインタラクションし，その結果から計算される累積報酬を用いて，価値を推定していたことになります．
（補足：ステップtでの状態の価値を推定するために，ステップt+1以降に得られる報酬が必要になります．そのため，REINFORCEでは，エピソード毎にしか方策を更新できません．）

TD誤差を用いた価値推定方法は，次のようなアイディアに基づいています．
まず，状態価値の再帰式に着目しましょう．
$$
V^{\pi}(s) = \mathrm{E}[r_{t+1} + \gamma V^{\pi}(s_{t+1}) \mid s_t = s]
$$
価値関数の推定値 $v_{\phi}(s)$ の目標値は$V^{\pi}(s)$となります．
すなわち，目標は$(V^{\pi}(s) - v_{\phi}(s))^2$を最小化することなどと解釈できます．
しかし，$V^{\pi}(s)$は未知なので，これを直接最適化することはできません．
上の再帰式における右辺の$V^{\pi}(s_{t+1})$も当然未知ですから，この右辺を直接使うこともできません．
しかし，$V^{\pi}(s_{t+1})$を現在の推定値$v_{\phi}(s_{t+1})$で近似することを許せば，
$$
V^{\pi}(s_t) \approx r_{t+1} + \gamma v_{\phi}(s_{t+1}) =: y_t
$$
と近似することができます．
そこで，上の近似式の右辺を$y_t$とおき，
$(y_{t} - v_{\phi}(s_t))^2$を最小化するように$\phi$を学習する方針を考えます．なお，$y_t$も$\phi$に依存していますが，こちらは定数と見なします．
このように，目標値を計算する際に推定値自身を利用する方法をブートストラップといい，この目標値との差$y_{t} - v_{\phi}(s_t)$をTD誤差と言います．
Actor-Critic法では，TD誤差を用いて状態価値関数を推定していきます．

価値推定の際，一点だけ注意が必要です．
状態$s_{t+1}$が終端状態である場合，すなわち，`terminated`フラグがTrueになっている場合，その状態の価値は$0$と解釈する必要があります．しかし，推定している価値関数は，終端状態について正しく学習されていません．
そこで，終端状態である場合には，
$$
V^{\pi}(s_t) \approx r_{t+1}
$$
とします．

## 方策勾配のオンライン推定

TD誤差を用いることで，各タイムステップで方策を更新することが可能になります．

第２回に紹介した，ベースラインとして状態価値を採用したREINFORCEアルゴリズムでは，各ステップでの方策勾配を以下のように推定していました．
$$
\left( G_{t} - v_{\phi}(s_{t}) \right) \nabla_{\theta} \ln \pi_\theta(a_{t} \mid s_{t})
$$
ここで，$G_t$は現状態より先のステップにおいて得られる報酬和ですから，ステップ$t$では計算できません．
$G_t$は状態$s_t$で行動$a_t$をとったときの行動価値の推定値として採用されており，価値関数の推定値を用いれば$y_t = r_{t+1} + \gamma v_{\phi}(s_{t+1})$で推定することが可能です．
すなわち，各ステップでの方策勾配を以下のように推定することが可能です．
$$
\left( y_t - v_{\phi}(s_{t}) \right) \nabla_{\theta} \ln \pi_\theta(a_{t} \mid s_{t})
$$


## バッチ更新とオンライン更新
TD誤差を用いる場合，現在よりも先の状態で得られる累積報酬を計算する必要が無いので，モンテカルロ法を用いる場合と異なり，各ステップで方策を更新していくことが可能です．
当然，一エピソード毎にパラメータ更新することも可能です．この場合をバッチ更新と呼ぶことにします．

バッチ更新の場合，Actor-Critic法とベースラインを推定するREINFORCEとでは，行動価値の推定方法のみが異なります．前者ではブートストラップを用いて行動価値を推定していますが，後者では累積報酬のモンテカルロ近似によって行動価値を推定しています．
モンテカルロ近似の良い点は，不偏推定になることですが，一般に分散が大きくなります．
他方，ブートストラップ推定する場合には，分散を抑えることができますが，一般に不偏となりません．


## Actor-Criticの実装

ここでは，まずバッチ学習（エピソード単位で学習）をするActor-Critic法を実装しています．ActorとCriticのアーキテクチャは第２回と同じものを採用しています．

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.distributions import Categorical


# gpuが使用される場合の設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

In [6]:
class Actor(nn.Module):
    def __init__(self, dim_state, num_action, dim_hidden=128):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(dim_state, dim_hidden)
        self.fc2 = nn.Linear(dim_hidden, num_action)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x

In [7]:
class Critic(nn.Module):
    def __init__(self, dim_state, dim_hidden=128):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(dim_state, dim_hidden)
        self.fc2 = nn.Linear(dim_hidden, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [28]:
class BatchActorCriticAgent:
    def __init__(self, env, actor, critic, device, lr_a, lr_c):
        self.device = device
        self.actor = actor
        self.critic = critic
        self.env = env
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_a, betas=(0.9, 0.999))
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_c, betas=(0.9, 0.999))

    def __call__(self, observation):
        return self.select_action(observation)[0]

    def select_action(self, observation):
        # 行動選択
        observation_ = Variable(torch.Tensor(observation)).to(self.device)
        action_probs = self.actor(observation_)
        log_probs = action_probs.log()
        action = Categorical(action_probs).sample()
        return action.data.cpu().numpy(), log_probs[action]

    def rollout_with_update(self):
        # 1 エピソード実行
        observation, info = self.env.reset()
        steps = 0
        l_observation = []
        l_next_observation = []
        l_reward = []
        l_terminated = []
        l_log_prob = []
        terminated = False
        truncated = False
        # エピソード
        while not (terminated or truncated):
            action, log_prob = self.select_action(observation)
            next_observation, reward, terminated, truncated, info = self.env.step(action)
            l_observation.append(observation)
            l_next_observation.append(next_observation)
            l_reward.append(reward)
            l_terminated.append(terminated)
            l_log_prob.append(log_prob)
            observation = next_observation
            steps +=1
        # 更新
        self.update(l_observation, l_next_observation, l_reward, l_terminated, l_log_prob)
        return l_reward

    def update(self, observation, next_observation, reward, terminated, log_prob):
        obs_tensor = torch.FloatTensor(np.array(observation)).to(self.device)
        next_obs_tensor = torch.FloatTensor(np.array(next_observation)).to(self.device)
        reward_tensor = torch.FloatTensor(np.array(reward)).reshape((-1, 1)).to(self.device)
        flg_tensor = torch.FloatTensor(np.array(terminated)).reshape((-1, 1)).to(self.device)

        vtt = (reward_tensor + (1 - flg_tensor) * self.critic(next_obs_tensor)).detach()
        vt = self.critic(obs_tensor)

        # Actor の更新
        loss_a = - sum([delta * lp for delta, lp in zip(vtt - vt.detach(), log_prob)]) / len(reward)
        self.actor_optimizer.zero_grad()
        loss_a.backward()
        self.actor_optimizer.step()

        # Critic の更新
        loss_c = torch.sum((vtt - vt)**2) / len(reward)
        self.critic_optimizer.zero_grad()
        loss_c.backward()
        self.critic_optimizer.step()

        return loss_a, loss_c

In [29]:
envname = "LunarLander-v2"
dim_state = 8
num_action = 4
env = gym.make(envname)

actor = Actor(dim_state = 8, num_action = 4).to(device)
critic = Critic(dim_state = 8).to(device)
agent = BatchActorCriticAgent(env, actor, critic, device, lr_a=2e-4, lr_c=2e-3)

In [30]:
interval = 100
returns = np.zeros((100, interval))

for i in range(returns.shape[0]):
    for j in range(returns.shape[1]):
        rewards = agent.rollout_with_update()
        returns[i, j] = np.sum(rewards)
    print(interval * (i+1), np.mean(returns[i]), np.std(returns[i]))

100 -206.72226816409108 127.67933144686761
200 -205.96241824425258 124.87223609715862
300 -213.06150953564105 127.43467492024337
400 -218.41733966373718 117.50842423462697
500 -268.98001157012516 162.43335674038389
600 -320.7105501212443 163.46326389224447
700 -286.85237903762896 150.47643598153584
800 -259.00883065239907 152.3478124305795
900 -227.72801756605054 128.01051892429743
1000 -192.24724545251857 113.49403416483617
1100 -168.86957584447322 107.51260418750971
1200 -150.33448305103806 106.72933037542533
1300 -148.12955373497206 110.32235219734953
1400 -146.05963672688017 95.80358465478132
1500 -136.40848976902706 86.79811229964129
1600 -138.32834751306055 95.6412481034117
1700 -141.02363146923253 92.62871410511903
1800 -102.23493721118803 78.28872389058175
1900 -105.84855892760822 91.13380988752857
2000 -104.47276601834699 84.6075949579003
2100 -98.38869862304811 89.76247670721541
2200 -99.87370111624979 96.96749185060983
2300 -94.46933205958175 89.70095374969394
2400 -81.22244

KeyboardInterrupt: ignored

In [ ]:
episodes = np.arange(1, 1+returns.size, returns.shape[1])
avg = np.mean(returns, axis=1)
std = np.std(returns, axis=1)
plt.errorbar(episodes, avg, std, linestyle=':', marker='^')
plt.grid()

経験分布関数についても確認しておきましょう．

In [ ]:
return_array = np.zeros(50)
for i in range(len(return_array)):
    history, img = rollout(envname, policy=agent, render=False)
    return_array[i] = cumulative_reward(history)

fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
ax.set_xlim(-400, 400)
plt.grid()

学習結果の確認は以下のコードで行います．

In [ ]:
history, img = rollout(envname, policy=agent, render=True)
print(cumulative_reward(history))
visualize(img)

## 連続タスクへの適用

REINFORCEを用いた場合，エピソディックタスクであることを仮定する必要がありました．
これは，価値をモンテカルロ推定しているため，エピソードが定義されていない場合これをうまく推定することができないことに起因しています．
TD誤差を用いる場合にはこの限りではありません．次状態の推定価値が得られれば，現状態についての行動価値が推定でき，方策勾配を計算できるためです．

## 割引累積報酬

これまでの議論ではエピソディックタスクを仮定していたため，エピソードが必ず有限のステップ$T$で終了し，累積報酬
$$
G_1 = \sum_{t=1}^{T} r_{t}
$$
が有限の値を取ることが仮定されてきました．そのため，減衰率$\gamma = 1$としてきました．
連続タスクを考える場合，即時報酬が有限の値であっても，累積報酬が発散してしまう可能性があります．
そこで，これに対する一つのアプローチとして，減衰率を$\gamma < 1$とした割引累積報酬
$$
G_1 = \sum_{t=1}^{T} \gamma^{t-1} r_{t}
$$
を考えることにします．こうすれば，即時報酬が有限である限り，割引累積報酬も有限となります．




In [74]:
class OnlineActorCriticAgent:
    def __init__(self, env, actor, critic, device, lr_a, lr_c):
        self.device = device
        self.actor = actor
        self.critic = critic
        self.env = env
        self.actor_optimizer = optim.Adam(self.actor.parameters(), lr=lr_a, betas=(0.9, 0.999))
        self.critic_optimizer = optim.Adam(self.critic.parameters(), lr=lr_c, betas=(0.9, 0.999))

    def __call__(self, observation):
        return self.select_action(observation)[0]

    def select_action(self, observation):
        # 行動選択
        observation_ = Variable(torch.Tensor(observation)).to(self.device)
        action_probs = self.actor(observation_)
        log_probs = action_probs.log()
        action = Categorical(action_probs).sample()
        return action.data.cpu().numpy(), log_probs[action]

    def rollout_with_update(self):
        # 1 エピソード実行
        observation, info = self.env.reset()
        steps = 0
        rewards = []
        terminated = False
        truncated = False
        while not (terminated or truncated):
            action, log_prob = self.select_action(observation)
            next_observation, reward, terminated, truncated, info = self.env.step(action)
            loss_a, loss_c = self.update(observation, next_observation, reward, terminated, log_prob)
            rewards.append(reward)
            observation = next_observation
            steps +=1
        return rewards

    def update(self, observation, next_observation, reward, terminated, log_prob):
        if terminated:
            vtt = reward
        else:
            vtt = reward + self.critic(torch.Tensor(next_observation).to(self.device)).detach()
        vt = self.critic(torch.Tensor(observation).to(self.device))

        # Actor の更新
        loss_a = - sum((vtt - vt.detach()) * log_prob)
        self.actor_optimizer.zero_grad()
        loss_a.backward()
        self.actor_optimizer.step()

        # Critic の更新
        loss_c = sum((vtt - vt)**2)
        self.critic_optimizer.zero_grad()
        loss_c.backward()
        self.critic_optimizer.step()

        return loss_a, loss_c

In [81]:
envname = "LunarLander-v2"
dim_state = 8
num_action = 4
env = gym.make(envname)

actor = Actor(dim_state = 8, num_action = 4).to(device)
critic = Critic(dim_state = 8).to(device)
agent = OnlineActorCriticAgent(env, actor, critic, device, lr_a=2e-6, lr_c=2e-5)

In [82]:
interval = 100
returns = np.zeros((100, interval))

for i in range(returns.shape[0]):
    for j in range(returns.shape[1]):
        rewards = agent.rollout_with_update()
        returns[i, j] = np.sum(rewards)
    print(interval * (i+1), np.mean(returns[i]), np.std(returns[i]))

100 -181.99175727453715 107.48923339347144
200 -207.41193447615248 131.59417374315046
300 -199.43683054483935 116.43834616185057
400 -184.1240167311555 107.16973285918472
500 -181.53275873818248 115.64225152678098
600 -197.6734166747809 118.16222983846815


KeyboardInterrupt: ignored

実際に実行してみるとすぐに気が付きますが，オンライン学習する場合，実行時間が長くかかります．
これは，ステップ毎にパラメータ更新を計算することが必要となるため，オーバーヘッドが多くかかるからです．
また，各ステップ方策勾配を計算する場合，計算される方策勾配の分散が大きくなるため，バッチ更新時に用いた学習率よりも小さめの学習率が必要になります．

# 自習課題

* 方策を変えてみましょう．特に，中間層のノード数を変更した場合に，学習効率がどの程度変わるのか，グラフを作成するなどして確認しましょう．

* 学習率を調整してみましょう．特に，ベースラインを導入したREINFORCEでは，Actorの学習率とCriticの学習率について，効率的なパラメータの関係を確認してみましょう．

* タスクを変えてみましょう．タスクが異なれば，適切な方策（ノード数など）や適切な学習率も変化する可能性があります．これを確認してみましょう．